In [5]:
import cvxpy as cp
import numpy as np
class ConvexProblemSolver:
    def __init__(self, n, K, M, B, epsilon_tree, J):
        self.n = n
        self.K = K
        self.M = M
        self.B = B
        self.epsilon_tree = epsilon_tree
        self.J = J

        

    def solve(self):
        # Define variables
        self.p = cp.Variable(self.n, pos=True)
 
        # Define expected value of tilde_L_n_minus_1
        expected_tilde_L_n_minus_1 = cp.sum([K**(n-1-q) * (K-1) * cp.prod([self.p[l] for l in range(q, n)]) for q in range(1,n)])

        # Define expected value of tilde_C_tree
        expected_tilde_C_tree = (n-1) * K + K * cp.sum([cp.sum([K**(j-q) * (K-1) * cp.prod([self.p[l] for l in range(q, j+1)]) for q in range(1,j+1)]) for j in range(1,n-2+1)])

        # Define objective function
        self.objective = cp.Minimize(expected_tilde_C_tree)

        # Define constraints
        self.constraints = [
            expected_tilde_L_n_minus_1 <= self.epsilon_tree,

            # cp.sum([cp.log(1/self.p[j])/cp.log(2) for j in range(1,n)]) == self.M - self.B,
            
            self.p >= 1/(2**self.J),
            self.p <= 1,
            self.p[0]==1
        ]
        # Define optimization problem
        problem = cp.Problem(self.objective, self.constraints)

        # Solve the problem
        problem.solve(gp='True')
        


        # Get optimal values
        optimal_p = self.p.value
        optimal_value = problem.value
        rounded_optimal_lengths = np.round(1/np.log(2)*np.log(1/optimal_p))
        # log(1/2^0) to the base 2 is 
        return optimal_p, optimal_value,rounded_optimal_lengths
 

# Example usage based on values in section 6 simulation results
n = 11  # Number of variables
K = 200  # Some value for K
B = 75   # Some value for B
epsilon_tree = 0.0069  # Some value for epsilon_tree
J = 15  # Some value for J
M = n*J # Some value for M

solver = ConvexProblemSolver(n, K, M, B, epsilon_tree, J)
optimal_p, optimal_value,lengths = solver.solve()

print("Optimal p:", optimal_p)
print("Optimal objective value:", optimal_value)
print("Lengths", lengths)


Optimal p: [1.00000000e+00 3.05176290e-05 3.05176287e-05 3.05176287e-05
 3.05176287e-05 3.05176287e-05 3.05176287e-05 3.05176287e-05
 3.05176287e-05 3.05176290e-05 3.07792336e-05]
Optimal objective value: 2010.9910398406914
Lengths [-0. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.]


In [3]:
import cvxpy as cp
print(cp.__version__)


1.4.1
